<a href="https://colab.research.google.com/github/Divyesh-Kanagavel/deep_learning--keras/blob/master/MNIST_basics_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook to understand deep learning basics - from scratch

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 6.6 MB/s 
     |████████████████████████████████| 451 kB 51.4 MB/s 
     |████████████████████████████████| 5.8 MB 40.3 MB/s 
     |████████████████████████████████| 1.3 MB 25.7 MB/s 
     |████████████████████████████████| 1.6 MB 46.6 MB/s 
     |████████████████████████████████| 132 kB 46.5 MB/s 
     |████████████████████████████████| 212 kB 16.2 MB/s 
     |████████████████████████████████| 182 kB 4.5 MB/s 
     |████████████████████████████████| 127 kB 21.5 MB/s 
     |████████████████████████████████| 7.6 MB 4.3 MB/s 


MessageError: ignored

In [ ]:
from fastai.vision.all import *

In [ ]:
from fastbook import *

In [ ]:
matplotlib.rc('image', cmap='Greys')

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
(path/'train').ls()

In [ ]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

In [ ]:
img3_path = threes[0]
img3 = Image.open(img3_path)
img3

In [ ]:
array(img3)[4:25,4:25]

In [ ]:
tensor(img3)[4:25,4:25]

In [ ]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]


In [ ]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255

In [ ]:
mean3 = stacked_threes.mean(0)
mean7 = stacked_sevens.mean(0)

In [ ]:
show_image(mean3)

In [ ]:
a_3 = stacked_threes[0]
show_image(a_3)

In [ ]:
a_7 = stacked_sevens[0]
show_image(a_7)

In [ ]:
dist_3_abs = (mean3-a_3).abs().mean()
dist_3_sqr = ((mean3-a_3)**2).mean().sqrt()

In [ ]:
dist_7_abs = (mean7 - a_7).abs().mean()
dist_7_sqr = ((mean7-a_7)**2).mean().sqrt()

In [ ]:
(dist_3_abs,dist_3_sqr), (dist_7_abs,dist_7_sqr)

In [ ]:
F.l1_loss(a_3.float(), mean3), F.l1_loss(a_7.float(),mean3)

In [ ]:
F.mse_loss(a_3,mean3).sqrt, F.mse_loss(a_7, mean7).sqrt

playing around with tensors and numpy arrays ..

In [ ]:
data = [[1,2],[4,5],[6,7]]
arr = np.array(data)
tnsr = tensor(data)


In [ ]:
arr

In [ ]:
tnsr

In [ ]:
tnsr.type()

In [ ]:
valid_3_tensor = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tensor = valid_3_tensor.float()/255.

In [ ]:
valid_3_tensor.shape

In [ ]:
valid_7_tensor = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tensor = valid_7_tensor.float()/255

In [ ]:
valid_7_tensor.shape

Calculation of mean average distance between ideal 3 and validation 3's and validation 7's

In [ ]:
def mnist_distance(a,b) : return (a-b).abs().mean((-1,-2))


Even if pass the validation set consisting of 1010 3's , the function would compute mnist_distance -> using a trick called broadcasting, where rank of smaller tensor is made to match the rank of larger tensor

In [ ]:
mnist_distance(mean3,valid_3_tensor)

In [ ]:
mnist_distance(mean3, valid_7_tensor)

In [ ]:
def is_3(x) : return mnist_distance(x,mean3) < mnist_distance(x,mean7)


In [ ]:
is_3(a_3), is_3(a_3).float()

In [ ]:
is_3(valid_3_tensor)

In [ ]:
accuracy_3 = is_3(valid_3_tensor).float().mean()
accuracy_7 = (1-is_3(valid_7_tensor).float()).mean()
accuracy_3, accuracy_7

Understand how gradients are calculated in pytorch automatically, reverseautodiff, autodiff etc

In [ ]:
def f(x) : return x**2

In [ ]:
xt = tensor(3.).requires_grad_()

In [ ]:
yt = f(xt)

In [ ]:
yt.backward()

In [ ]:
xt.grad

the gradient is correctly computed as 2*x

an end-end SGD example to do learning for a functin

In [ ]:
time = torch.arange(0,20).float(); time

In [ ]:
speed = 0.75*(time-9.5)**2 + 1 + torch.randn(20)*3
plt.scatter(time,speed)

random gaussian noise added to the function to make the scenario real

In [ ]:
# an approximate quadratic function with parameters a,b,c are to be found to fit the data

def f(t,params):
  a,b,c = params
  return a*(t**2) + b*t + c 

In [ ]:
def mse(preds,targets) : return ((preds-targets)**2).mean() 

1. Initialize the parameters

In [ ]:
params = torch.randn(3).requires_grad_()
orig_params = params.clone()

2. Calculate prediction


In [ ]:
preds = f(time,params)

In [ ]:
def show_preds(preds, ax=None):
    if ax is None: ax=plt.subplots()[1]
    ax.scatter(time, speed)
    ax.scatter(time, to_np(preds), color='red')
    ax.set_ylim(-300,100)

In [ ]:
show_preds(preds)

In [ ]:
loss = mse(preds, speed)

In [ ]:
loss

In [ ]:
loss.backward()

In [ ]:
params.grad

In [ ]:
params

In [ ]:
params.data

adjust the gradients with learning rate and set the weights for this iteration

In [ ]:
lr = 1e-5
params.data -= lr*params.grad.data
params.grad = None

In [ ]:
preds = f(time,params)

In [ ]:
mse(preds,speed)

In [ ]:
show_preds(preds)

let us make a function out of this to run it mulitple times...

In [ ]:
def apply_step(params,prn=True):
  preds = f(time,params)
  loss = mse(preds, speed)
  loss.backward()
  params.data -= lr*params.grad.data
  params.grad = None
  if prn : print(loss.item())
  return preds

Repeat the loop

In [ ]:
for i in range(10): apply_step(params)

In [ ]:
params

In [ ]:
orig_params

In [ ]:
params = orig_params.detach().requires_grad_()

In [ ]:
params

In [ ]:
orig_params

MNIST dataset, perform the same set of processes

In [ ]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1,28*28)

In [ ]:
train_x.shape


view function, reshapes the dataset with the final axis fitting 28*28 values

In [ ]:
train_y = tensor([1]*len(threes)+[0]*len(sevens)).unsqueeze(1)
train_y.shape

In [ ]:
dset = list(zip(train_x,train_y))

In [ ]:
x,y = dset[0]
x.shape,y.shape

In [ ]:
valid_x = torch.cat([valid_3_tensor,valid_7_tensor]).view(-1,28*28)
valid_y = torch.tensor([1]*len(valid_3_tensor)+[0]*len(valid_7_tensor)).unsqueeze(1)
valid_x.shape,valid_y.shape

In [ ]:
valid_dset = list(zip(valid_x,valid_y))

Seven step -learning process to be begun:
Step 1 : Initialize the parameters



In [ ]:
def init_params(size, std=1.0) : return (torch.randn(size)*std).requires_grad_()

In [ ]:
weights = init_params((28*28,1))

In [ ]:
bias = init_params(1)

In [ ]:
(train_x[0]*weights.T).sum() + bias

In [ ]:
#Vectorized operation

def linear1(xb) : return xb@weights + bias
preds = linear1(train_x)

In [ ]:
preds


In [ ]:
corrects = (preds > 0.0).float() == train_y

In [ ]:
corrects.float().mean().item()

In [ ]:
def mnist_loss(preds,targets) : return torch.where(targets == 1, 1-preds, preds).mean()

In [ ]:
help(torch.where)

Activation function : Sigmoid - to squash the output to the range 0 to 1

In [ ]:
def sigmoid(x) : return 1/(1+torch.exp(-x))

In [ ]:
plot_function(torch.sigmoid, title='Sigmoid',min=-4,max=-4)

In [ ]:
def mnist_loss(preds,targets):
  preds = preds.sigmoid()
  return torch.where(targets == 1, 1-preds,preds).mean()

DataLoader class : fastai implementation of a class which takes any collection of data into data structure with an iterator

In [ ]:
coll = range(15)
dl = DataLoader(coll,batch_size=5,shuffle=True)

In [ ]:
list(dl)

Dataset in pytorch  : Collection of tuples containing dependent and independent variables - example given below

In [ ]:
ds = L(enumerate(string.ascii_lowercase))

In [ ]:
ds

In [ ]:
dl = DataLoader(ds,batch_size=6, shuffle=True)
list(dl)

the process of SGD for MNIST dataset : from scratch using minimal Python libraries

In [ ]:
weights = init_params((28*28,1))

In [ ]:
bias = init_params(1)

In [ ]:
dl = DataLoader(dset,batch_size=256)
xb,yb = first(dl)
xb.shape,yb.shape

In [ ]:
valid_dl = DataLoader(valid_dset,batch_size=256)


In [ ]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [ ]:
def train_epoch(model, lr, params):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [ ]:
def batch_accuracy(xb,yb):
  preds = xb.sigmoid()
  correct = (preds>0.5) == yb

 
  return correct.float().mean()

In [ ]:
def validate_epoch(model):
  accs = [batch_accuracy(model(x),y) for x,y in valid_dl]
  return round(torch.stack(accs).mean().item(),4)
  

In [ ]:
validate_epoch(linear1)

In [ ]:
lr = 1
params = weights,bias
train_epoch(linear1,lr,params)
validate_epoch(linear1)

In [ ]:
for i in range(20):
  train_epoch(linear1,lr,params)
  print(validate_epoch(linear1),end=' ')

Do the same thing with pytorch's modules -> no need to reinvent the wheel

In [ ]:
linear_model = nn.Linear(28*28,1)

In [ ]:
w,b = linear_model.parameters()
w.shape,b.shape

In [ ]:
b


In [ ]:
class BasicOptim:
  def __init__(self,params,lr) : self.params,self.lr = params, lr

  def step(self,*args,**kwargs):
    for p in self.params : p.data -= p.grad.data*self.lr

  def zero_grad(self,*args,**kwargs):
    for p in self.params : p.grad = None





In [ ]:
lr = 1

In [ ]:
opt = BasicOptim(linear_model.parameters(), lr)

In [ ]:
def train_epoch(model):
  for xb,yb in dl:
    calc_grad(xb,yb,model)
    opt.step()
    opt.zero_grad()

In [ ]:
validate_epoch(linear_model)

In [ ]:
def train_model(model,epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model),end=' ')

In [ ]:
train_model(linear_model,30)

Even these steps could be done with fastai's SGD class, encapsulates all these modules inside it.

In [ ]:
linear_model = nn.Linear(28*28,1)
opt = SGD(linear_model.parameters(),lr)
train_model(linear_model,30)

Fastai has its own Learner class which accepts all the configuration parameters along with the Dataloader consiting of training dataloader & validation dataloader

In [ ]:
dls = DataLoaders(dl,valid_dl)


In [ ]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(10, lr=lr)


In [ ]:
  plot_function(F.relu)

In [ ]:
#pytorch functionality to construct a network quickly

simple_net =  nn.Sequential(nn.Linear(28*28,30), nn.ReLU(),nn.Linear(30,1))

In [ ]:
learn = Learner(dls, simple_net, opt_func=SGD, loss_func = mnist_loss, metrics = batch_accuracy)


In [ ]:
learn.fit(40,0.1)

In [ ]:
plt.plot(L(learn.recorder.values).itemgot(2))

In [ ]:
learn.recorder.values

In [ ]:
learn.recorder.values[-1][2]

Practice shows that neural network with more number of layers with small number of neurons per layer is more effective than less number of layers with large number of neurons per layer